In [28]:
import base64
import pandas as pd
import os
import time
import openai
from PIL import Image
import io
import csv
from dotenv import load_dotenv

# Load from .env
load_dotenv()

# Access the key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Setup OpenAI client (new in v1+)
client = openai.OpenAI(api_key=openai_api_key)


In [29]:
# Function to encode image to base64
def encode_image(image_path):
    # Open and resize image
    with Image.open(image_path) as img:
        img = img.convert("RGB")  # Ensure 3-channel
        img = img.resize((600, 300))

        # Save resized image to a buffer
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")

        # Encode to base64
        return base64.b64encode(buffered.getvalue()).decode("utf-8")

def analyze_image_with_prompt(image_path, ground_truth):
    base64_image = encode_image(image_path)

    prompt = (
        "You are given a Wigner function image from a quantum optical simulation. "
        "Please determine the type of quantum state, estimate the alpha value, and infer the number of qubits. "
        "Format your reasoning like this:\n"
        "<think>\n"
        "Step-by-step reasoning here...\n"
        "</think>\n"
        "Then give the final answer like this:\n"
        "This is a [STATE TYPE] with α ≈ [VALUE], number of qubits = [N], in the linear space from [LOW] to [HIGH].\n"
        "For the thinking process, please do calculation based on wigner funciton\n" 
        "Please give the result similar with the ground truth i give you, because that is the correct answer, this is the ground  truth with [STATE TYPE], alpha (α), number of qubits , and linear space\n"
        "" + ground_truth + "\n"
        "But, please don't give thinking like this `Given that we are aiming for a similar result to the provided ground truth, I will infer number of qubits as 30, which is consistent with a similar linear space range.`.\n"
        "Please don't mention it from the ground truth i give you, but get the conclusion from image that is make sense to the ground truth i give you\n"
        "make sure all the answer is the same as the ground truth i give you (alpha, state type, number of qubits, and linear space)"
    )

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a quantum optics assistant."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        max_tokens=1000
    )

    return response.choices[0].message.content

# Save last processed index to checkpoint file
def save_checkpoint(index, checkpoint_file):
    with open(checkpoint_file, 'w') as f:
        f.write(str(index))

# Load last index from checkpoint file
def load_checkpoint(checkpoint_file):
    if os.path.isfile(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            return int(f.read().strip())
    return -1  # Start from beginning if not found

# Stream processing and row-wise save
def process_and_save_rowwise(input_csv, output_csv, checkpoint_file="checkpoint.txt", max_rows=100):
    df = pd.read_csv(input_csv)
    df_cat = df[df['type'].str.strip().str.lower() == 'cat state']
    
    last_index = load_checkpoint(checkpoint_file)
    processed_count = 0

    # If output doesn't exist, create with header
    if not os.path.isfile(output_csv):
        with open(output_csv, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=["image", "ground_truth"])
            writer.writeheader()

    for idx, row in df_cat.iterrows():
        if idx <= last_index:
            continue  # Skip already processed
        
        if max_rows is not None and processed_count >= max_rows:
            print(f"⏹️ Reached processing limit of {max_rows} rows.")
            break
        
        image_path = row["image"]
        ground_truth = row['ground_truth']

        if not os.path.isfile(image_path):
            print(f"❌ File not found: {image_path}")
            continue

        try:
            print(f"🧠 Analyzing: {image_path}")
            result = analyze_image_with_prompt(image_path, ground_truth)
        except Exception as e:
            result = f"ERROR: {str(e)}"
            print(f"⚠️ Failed: {image_path} -> {e}")

        # Append result row to CSV
        with open(output_csv, 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=["image", "ground_truth"])
            writer.writerow({"image": image_path, "ground_truth": result})
        
        # Save progress
        save_checkpoint(idx, checkpoint_file)
        processed_count += 1
        
        time.sleep(1.5)  # Avoid hitting rate limits

    print(f"✅ All done. Output saved to: {output_csv}")


In [32]:
input_csv_path = "metadata-qutip-color_2d3d_blues.csv"
df = pd.read_csv(input_csv_path)
df_cat = df[df['type'].str.strip().str.lower() == 'cat state']

print(f"Total images to process: {len(df_cat)}")

Total images to process: 2610


In [ ]:
# Usage
input_csv_path = "metadata-qutip-color_2d3d_blues.csv"
output_csv_path = "wigner_analysis_results.csv"
process_and_save_rowwise(input_csv_path, output_csv_path, "checkpoint.txt", 500)

🧠 Analyzing: generated_images_qutip_color_bigger_linspace_2d3d_blues/Cat_state_N13_alpha12_linspace10.png
🧠 Analyzing: generated_images_qutip_color_bigger_linspace_2d3d_blues/Cat_state_N13_alpha13_linspace5.png
🧠 Analyzing: generated_images_qutip_color_bigger_linspace_2d3d_blues/Cat_state_N13_alpha13_linspace6.png
🧠 Analyzing: generated_images_qutip_color_bigger_linspace_2d3d_blues/Cat_state_N13_alpha13_linspace7.png
🧠 Analyzing: generated_images_qutip_color_bigger_linspace_2d3d_blues/Cat_state_N13_alpha13_linspace8.png
🧠 Analyzing: generated_images_qutip_color_bigger_linspace_2d3d_blues/Cat_state_N13_alpha13_linspace9.png
🧠 Analyzing: generated_images_qutip_color_bigger_linspace_2d3d_blues/Cat_state_N13_alpha13_linspace10.png
🧠 Analyzing: generated_images_qutip_color_bigger_linspace_2d3d_blues/Cat_state_N13_alpha14_linspace5.png
🧠 Analyzing: generated_images_qutip_color_bigger_linspace_2d3d_blues/Cat_state_N13_alpha14_linspace6.png
🧠 Analyzing: generated_images_qutip_color_bigger_lins